In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!cp -r /content/drive/MyDrive/Data /content/

In [ ]:
!unzip  Data/cloudasn.zip
!unzip  Data/cdnasn.zip
!unzip  Data/vpnasn.zip
!unzip  Data/hostingasn.zip

Archive:  Data/cloudasn.zip
  inflating: cloudasn.csv            
Archive:  Data/cdnasn.zip
  inflating: cdnasn.csv              
Archive:  Data/vpnasn.zip
  inflating: vpnasn.csv              
Archive:  Data/hostingasn.zip
  inflating: hostingasn.csv          


In [ ]:
!pip install html2text

In [ ]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [ ]:
import html2text
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
sample_size = 35000
df = pd.read_csv('cdnasn.csv')
df['vendor']= ['cdn' for x in range(df.shape[0])]
df = df.sample(n = sample_size)
df1 = pd.read_csv('cloudasn.csv')
df1['vendor']= ['cloud' for x in range(df1.shape[0])]
df1 = df1.sample(n = sample_size)
df2 = pd.read_csv('hostingasn.csv')
df2['vendor']= ['hosting' for x in range(df2.shape[0])]
df2 = df2.sample(n = sample_size)
df3 = pd.read_csv('vpnasn.csv')
df3['vendor']= ['vpn' for x in range(df3.shape[0])]
df3 = df3.sample(n = sample_size)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (11,26) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
df.columns

Index(['ip', 'data.http.result.response.headers.server.0',
       'data.http.result.response.status_code',
       'data.http.result.response.headers.unknown.0.key',
       'data.http.result.response.headers.unknown.0.value.0',
       'data.http.result.response.headers.connection.0',
       'data.http.result.response.headers.via.0',
       'data.http.result.response.headers.unknown.1.key',
       'data.http.result.response.headers.unknown.1.value.0',
       'data.http.result.response.headers.accept_ranges.0',
       'data.http.result.response.headers.cache_control.0',
       'data.http.result.response.headers.retry_after.0',
       'data.http.result.response.body',
       'data.http.result.response.body_sha256',
       'data.http.result.response.content_length',
       'data.http.result.response.headers.content_type.0',
       'data.http.result.response.headers.location.0',
       'data.http.result.redirect_response_chain.0.status_line',
       'data.http.result.response.headers.etag.0'

In [ ]:
df = df.append([df1,df2,df3],ignore_index=True)

In [ ]:
df = df[['data.http.result.response.body', 'vendor']]

In [ ]:
bodylist = df['data.http.result.response.body'].values

In [ ]:
h = html2text.HTML2Text()
h.ignore_links = True

In [ ]:
corpuslist = []

for i in bodylist:
   corpuslist.append(h.handle(str(i)))

In [ ]:
df2['data.http.result.response.body'].value_counts()

<!doctype html>\n<html>\n\n\t<head>\n\t\t<title>Site not found &middot; DreamHost</title>\n\t\t<meta http-equiv="cache-control" content="no-cache" />\n\t\t<meta name="description" content="The owner of this domain has not yet uploaded their website." />\n\t\t<link rel="stylesheet" href="//d1a6zytsvzb7ig.cloudfront.net/newpanel/css/singlepage.css" />\n\t</head>\n\n\t<body>\n\t\t<div class="page page-missing">\n\t\t\t<h1>Site Not Found</h1>\n\n\t\t\t<p>Well, this is awkward. The site you're looking for is not here.</p>\n\t\t\t<p><small>Is this your site? <a href="https://help.dreamhost.com/hc/en-us/articles/215613517"\n\t\t\t\t\t\trel="nofollow">Get more info</a> or <a\n\t\t\t\t\t\thref="https://panel.dreamhost.com/index.cgi?tree=support.msg" rel="nofollow">contact\n\t\t\t\t\t\tsupport</a>.</small></p>\n\n\t\t\t<a href="https://www.dreamhost.com/" class="logo" rel="nofollow">DreamHost</a>\n\t\t</div>\n\t</body>\n\n</html>\n                                                                 

In [ ]:
df3['data.http.result.response.body'].value_counts()

<html>\r\n<head><title>301 Moved Permanently</title></head>\r\n<body>\r\n<center><h1>301 Moved Permanently</h1></center>\r\n</body>\r\n</html>\r\n                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [ ]:
df[df["vendor"] == 'hosting'].count

<bound method DataFrame.count of                            data.http.result.response.body   vendor
70000                                                 NaN  hosting
70001   <!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//...  hosting
70002   <html>\r\n<head><title>301 Moved Permanently</...  hosting
70003                                                 NaN  hosting
70004   <!doctype html>\n<html>\n\n\t<head>\n\t\t<titl...  hosting
...                                                   ...      ...
104995  <!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Fi...  hosting
104996                                                NaN  hosting
104997  <meta http-equiv="refresh" content="0;url=http...  hosting
104998                                                NaN  hosting
104999                                                NaN  hosting

[35000 rows x 2 columns]>

In [ ]:
vectorizer = TfidfVectorizer(min_df=100,lowercase=True,stop_words='english',token_pattern=u'(?ui)\\b\\w*[a-z]+\\w*\\b')
X = vectorizer.fit_transform(corpuslist)

In [ ]:
X_names = vectorizer.get_feature_names()

In [ ]:
len(X_names)

2385

In [ ]:
X_names

In [ ]:
tf_df = pd.DataFrame(X.toarray(),columns=X_names)

In [ ]:
tf_df['vendor'] = df['vendor']

In [ ]:
tf_df['vendor'] = le.fit_transform(tf_df['vendor'].astype(str))

In [ ]:
vendor_encoded = tf_df['vendor']

In [ ]:
tf_df = tf_df.drop(['vendor'],axis=1)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(tf_df, vendor_encoded, test_size=0.4, random_state=42)

In [ ]:
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X_train, y_train) 
y_pred = clf.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test,y_pred))

[[13961     0     0     0]
 [ 1437 12194   420     0]
 [   85     1 13927     0]
 [  319     1   359 13296]]
              precision    recall  f1-score   support

           0       0.88      1.00      0.94     13961
           1       1.00      0.87      0.93     14051
           2       0.95      0.99      0.97     14013
           3       1.00      0.95      0.98     13975

    accuracy                           0.95     56000
   macro avg       0.96      0.95      0.95     56000
weighted avg       0.96      0.95      0.95     56000

0.9531785714285714


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=2)
neigh.fit(X_train, y_train)
y_pred = neigh.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test,y_pred))

[[13570   390     1     0]
 [  261 13649   118    23]
 [  167   236 13565    45]
 [   83   405   276 13211]]
              precision    recall  f1-score   support

           0       0.96      0.97      0.97     13961
           1       0.93      0.97      0.95     14051
           2       0.97      0.97      0.97     14013
           3       0.99      0.95      0.97     13975

    accuracy                           0.96     56000
   macro avg       0.97      0.96      0.96     56000
weighted avg       0.96      0.96      0.96     56000

0.9641964285714286


In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
clf = LogisticRegression(random_state=0,max_iter=200).fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test,y_pred))

[[13940    20     0     1]
 [  232 13748    69     2]
 [   77    36 13894     6]
 [  307    19   363 13286]]
              precision    recall  f1-score   support

           0       0.96      1.00      0.98     13961
           1       0.99      0.98      0.99     14051
           2       0.97      0.99      0.98     14013
           3       1.00      0.95      0.97     13975

    accuracy                           0.98     56000
   macro avg       0.98      0.98      0.98     56000
weighted avg       0.98      0.98      0.98     56000

0.9797857142857143
